In [9]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import csv

In [3]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [20]:
import urllib.request

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req = urllib.request.urlopen(url)
article = req.read().decode()

with open('List_of_postal_codes_of_Canada.html', 'w') as fo:
    fo.write(article)
    

In [47]:
from bs4 import BeautifulSoup

# Load article, turn into soup and get the <table>s.
article = open('List_of_postal_codes_of_Canada.html').read()
soup = BeautifulSoup(article, 'html.parser')
tables = soup.find_all('table', class_='sortable')

# Search through the tables for the one with the headings we want.
for table in tables:
    ths = table.find_all('th')
    headings = [th.text.strip() for th in ths]
    if headings[:3] == ['Postcode', 'Borough', 'Neighbourhood']:
        break

# Extract the columns we want and write to a semicolon-delimited text file.
with open('List_of_postal_codes_of_Canada.csv', 'w') as fo:
    for tr in table.find_all('tr'):
        tds = tr.find_all('td')
        if not tds:
            continue
        postcode, borough, neighbourhood = [td.text.strip() for td in tds[:3]]
        print(', '.join([postcode, borough, neighbourhood]), file=fo)

In [66]:
colnames=['Postcode', 'Borough', 'Neighbourhood']
df = pd.read_csv('List_of_postal_codes_of_Canada.csv',names=colnames)
df['Borough']=df['Borough'].str.strip()
df['Neighbourhood'] = df['Neighbourhood'].str.strip()
df.shape

(287, 3)

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.


In [64]:
df1 = df[df['Borough'] != 'Not assigned']
df1.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [65]:
df1.shape

(210, 3)

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [71]:
df2=df1.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
df2.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [72]:
df2.shape

(103, 3)

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

